In [ ]:

import pandas as pd

# Assuming your JSON data is stored in a file named 'data.json'
# If you have your JSON data as a string or in a different format, this step may vary
json_file_path = 'catalog.json'
df = pd.read_json(json_file_path)

# Group items by 'Type' and aggregate information. For demonstration, we'll just list the Names of the items.
# You can customize this aggregation to include other operations like counting items, averaging prices, etc.
grouped_items = df.groupby('Type').agg({'Name': lambda x: list(x)})

print(grouped_items)



TODO create a function to create data for Actual Ratings

In [16]:
import json
import random


# Load the product catalog to get product IDs
with open("./data/catalog/catalog_w_text3_emb.json", "r") as file:
    product_catalog = json.load(file)

# Extract product IDs
product_ids = [product["Id"] for product in product_catalog]

# Define a maximum number of products a user can rate
max_products_per_user = 20

# Assuming a max rating value
max_rating = 10

# Generate user IDs from 1 to 1001
current_user_ids = [144, 496, 189, 232, 194, 950, 370, 980, 190, 404, 737, 959, 142, 795, 121, 743, 307, 365, 30, 726, 339, 536]

current_ratings = []
# Generate new ratings while ensuring not all products are rated
for user_id in current_user_ids:
    # Randomly select a subset of products to rate, respecting the max_products_per_user limit
    products_to_rate = random.sample(product_ids, min(len(product_ids), random.randint(1, max_products_per_user)))
    
    for product_id in products_to_rate:
        product_info = next((product for product in product_catalog if product["Id"] == product_id), None)
        if product_info:  # If product found
            new_rating = {
                "UserId": user_id,
                "ProductId": product_id,
                "Rating": random.randint(1, max_rating)
            }
            current_ratings.append(new_rating)

# Convert the updated data to JSON format
json_data = json.dumps(current_ratings, indent=4)

# Write the updated JSON data to a new file
with open("./data/ratings/actualRatings.json", "w") as file:
    file.write(json_data)


## Randomly rates 20 products for 1000 users 


In [13]:
import json
import random


# Load the product catalog to get product IDs
with open("./data/catalog/catalog_w_text3_emb.json", "r") as file:
    product_catalog = json.load(file)

# Extract product IDs
product_ids = [product["Id"] for product in product_catalog]

# Define a maximum number of products a user can rate
max_products_per_user = 20

# Assuming a max rating value
max_rating = 10

# Define the number of user IDs you want
num_user_ids = 10001

# Generate user IDs from 1 to 1001
current_user_ids = list(range(1, num_user_ids))

current_ratings = []
# Generate new ratings while ensuring not all products are rated
for user_id in current_user_ids:
    # Randomly select a subset of products to rate, respecting the max_products_per_user limit
    products_to_rate = random.sample(product_ids, min(len(product_ids), max_products_per_user))
    
    for product_id in products_to_rate:
        product_info = next((product for product in product_catalog if product["Id"] == product_id), None)
        if product_info:  # If product found
            new_rating = {
                "UserId": user_id,
                "ProductId": product_id,
                "Rating": random.randint(1, max_rating)
            }
            current_ratings.append(new_rating)

# Convert the updated data to JSON format
json_data = json.dumps(current_ratings, indent=4)

# Write the updated JSON data to a new file
with open("./data/ratings/randomRatings.json", "w") as file:
    file.write(json_data)


## Display the number of items rated by each user

In [ ]:
import json

# Load the updated ratings data
with open("./data/ratings/actualRatings.json", "r") as file:
    ratings = json.load(file)

# Count the number of items rated by each user
ratings_count_by_user = {}
for rating in ratings:
    user_id = rating["UserId"]
    if user_id in ratings_count_by_user:
        ratings_count_by_user[user_id] += 1
    else:
        ratings_count_by_user[user_id] = 1

total_user_ids = len(ratings_count_by_user)

# Print the total number of user IDs
print(f"Total number of user IDs: {total_user_ids}")

for user_id, count in ratings_count_by_user.items():
    print(f"User ID: {user_id} has rated {count} items.")




## Advanced Rating system
Now taking in random ratings and giving it a pattern based on brand/ type/ price. Storing it in a new file 'AugmentedRating'


In [15]:
import json
import random

# Load the current user ratings
with open("./data/ratings/randomRatings.json", "r") as file:
    current_ratings = json.load(file)

# Load the product catalog
with open("./data/catalog/catalog_w_text3_emb.json", "r") as file:
    product_catalog = json.load(file)

total_desired_ratings = 200000
# Define preferences
brand_preferences = {"Daybird": 8, "Gravitator": 7, "WildRunner": 9, "Quester": 6, "B&R": 5, "Raptor Elite": 7, "Solstix": 8, "Grolltex": 6, "AirStrider": 7, "Green Equipment": 9, "Legend": 5, "Zephyr": 6, "XE": 4}
type_preferences = ["Footwear", "Climbing", "Ski/boarding", "Bags", "Jackets", "Navigation", "Cycling", "Trekking"]

curr = range(1, 10001)
# Simulate user preferences for product types
user_type_preferences = {user_id: random.sample(type_preferences, k=random.randint(1, len(type_preferences))) for user_id in curr}

# Normalize price to a 1-10 scale
max_price = max(product["Price"] for product in product_catalog)
min_price = min(product["Price"] for product in product_catalog)
price_range = max_price - min_price

def normalize_price(price):
    return 1 + (((price - min_price) / price_range) * 9)

# Function to calculate rating based on brand, type, and price
def calculate_rating(user_id, product):
    price_score = normalize_price(product["Price"])
    brand_score = brand_preferences.get(product["Brand"], 5)  # Default to mid-preference
    type_score = 10 if product["Type"] in user_type_preferences[user_id] else 5
    # Weighted average of the scores
    return (price_score * 0.4 + brand_score * 0.3 + type_score * 0.3)

current = []
# Generate new ratings considering user preferences
for user_id in curr:  # Limit to 20 different users for this example
    for product in product_catalog:
        if (user_id, product["Id"]) in user_product_pairs:  # Skip if already rated
            continue
        # Calculate new rating based on sophisticated algorithm
        rating = calculate_rating(user_id, product)
        new_rating = {
            "UserId": user_id,
            "ProductId": product["Id"],
            "Rating": round(rating, 2)  # Rounded for readability
        }
        current.append(new_rating)
        if len(current) >= total_desired_ratings:  # Check if desired total reached
            break
    if len(current) >= total_desired_ratings:
        break

# Convert the updated data to JSON format
json_data = json.dumps(current, indent=4)

with open("./data/ratings/AugmentedRating.json", "w") as file:
    file.write(json_data)
